<a href="https://colab.research.google.com/github/oscarWoj/CISC499/blob/main/499_bertopic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Refer to this if you have any questions: https://towardsdatascience.com/interactive-topic-modeling-with-bertopic-1ea55e7d73d8

## Data Preprocessing

Data will be downloaded from google drive folder

In [1]:
!pip install numpy~=1.20.0
!pip install bertopic
!sudo apt-get install unzip
!pip install gdown
!pip install --upgrade gdown


Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.2).
0 upgraded, 0 newly installed, 0 to remove and 193 not upgraded.


In [2]:
#!pip install llvmlite --ignore-installed
#!pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
#!pip install cuml-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
#!pip install cugraph-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
#!pip uninstall cupy-cuda115 -y
#!pip uninstall cupy-cuda11x -y
#!pip install cupy-cuda11x -f https://pip.cupy.dev/aarch64

#!pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
#!pip install cuml-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
#!pip install cugraph-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
#!pip uninstall cupy-cuda115 -y
#!pip uninstall cupy-cuda11x -y
#!pip install cupy-cuda11x -f https://pip.cupy.dev/aarch64

In [3]:
#from cuml.cluster import HDBSCAN
#from cuml.manifold import UMAP

In [4]:
import torch
import csv
import gdown
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

from sklearn.decomposition import PCA
import numpy as np

In [5]:
from umap import UMAP

In [6]:
#device = ‘cuda’ if torch.cuda.is_available() else ‘cpu’

In [7]:
torch.cuda.is_available()

True

### You can skip the next 2 lines if data is already in

In [8]:
#!gdown -v --fuzzy https://drive.google.com/file/d/1Xtc9eTSyjQEaGoRgU9VGlz6YB4SvPoMc/view?usp=sharing
#!gdown --fuzzy https://drive.google.com/file/d/1KvbaoIV18t7bMboOAJfkgyrf5ENGiSWH/view?usp=share_link
!gdown --fuzzy https://drive.google.com/file/d/1RmIhftc9tRw4MMC_f_oSdhPsVb1NoqpZ/view?usp=share_link

Downloading...
From: https://drive.google.com/uc?id=1RmIhftc9tRw4MMC_f_oSdhPsVb1NoqpZ
To: /home/jovyan/project/CISC499/output_complex.csv.zip
100%|████████████████████████████████████████| 784M/784M [00:10<00:00, 73.9MB/s]


In [9]:
!unzip -vo ~/project/CISC499/output_complex.csv.zip
#!unzip ~/project/CISC499/output_simple.csv.zip
#!unzip ~/project/CISC499/outputfile.csv.zip

Archive:  /home/jovyan/project/CISC499/output_complex.csv.zip
 Length   Method    Size  Cmpr    Date    Time   CRC-32   Name
--------  ------  ------- ---- ---------- ----- --------  ----
2710034252  Defl:N 784241998  71% 2023-02-15 04:47 32926c1e  output_complex.csv
--------          -------  ---                            -------
2710034252         784241998  71%                            1 file


### Reading Data

In [8]:
data = []
with open(r'output_complex.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row[0])

In [9]:
data[8]

'a lot of big website use cloaking too   i have heard amazon com uses cloaking '

In [10]:
#data[8][2]

In [11]:
#parsed_data=[]


In [12]:
data[0]

'text'

## Optimizing and running the model

## change to CPU only - Optimization -1 

In [11]:
#from sklearn.pipeline import make_pipeline
#from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import TfidfVectorizer

#pipe = make_pipeline(
 #   TfidfVectorizer(),
 #   TruncatedSVD(100))

#topic_model_nocpu = BERTopic(embedding_model=pipe)

### Precompute Embeddings - Optimization #1

In [13]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

In [14]:

embeddings = sentence_model.encode(data[1:], show_progress_bar=True)
# Takes ~ 40 Mins w/ max cluster settings

Batches:   0%|          | 0/311751 [00:00<?, ?it/s]

In [14]:
#Embeddings sample
embeddings_sample = sentence_model.encode(data[1:len(data)//4], show_progress_bar=True)
# Takes ~ 20 min w/ max cluster settings

Batches:   0%|          | 0/155875 [00:00<?, ?it/s]

### Speed up umap - Optimization #2

In [15]:
def rescale(x, inplace=False):
    """ Rescale an embedding so optimization will not have convergence issues.
    """
    if not inplace:
        x = np.array(x, copy=True)

    x /= np.std(x[:, 0]) * 10000

    return x

In [18]:
# Initialize and rescale PCA embeddings
pca_embeddings = rescale(PCA(n_components=5).fit_transform(embeddings))

In [16]:
pca_embeddings_sample = rescale(PCA(n_components=5).fit_transform(embeddings_sample))

In [19]:
# Start UMAP from PCA embeddings
umap_model = UMAP(
    n_neighbors=15,
    n_components=5, #5
    min_dist=0.0,
    metric="cosine",
    init=pca_embeddings,
    low_memory=True
)

In [17]:
umap_model_sample = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    init=pca_embeddings_sample,
    low_memory=True
)

### Putting it all together

In [18]:
sample = True

In [19]:
!date

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Mar  6 16:54:31 UTC 2023


In [20]:
#train on only cpu - lighweight
#if sample:
#    topic_model_nogpu = BERTopic(embedding_model=pipe,umap_model=umap_model_sample,verbose=True,min_topic_size=100,n_gram_range=(1,2))
#    topics, probs = topic_model_nogpu.fit_transform(data[1:len(data)//2])
#else:
#    topic_model_nogpu = BERTopic(embedding_model=pipe,umap_model=umap_model,verbose=True,min_topic_size=100,n_gram_range=(1,2))
#    topics, probs = topic_model_nogpu.fit_transform(data[1:])

In [21]:
!date

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Mar  6 16:54:31 UTC 2023


In [22]:
!date

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Mar  6 16:54:32 UTC 2023


In [23]:
#train on both (using gpu isnt necessarily "correct" on jupyterlab instance)
if sample:
    topic_model_gpu = BERTopic(umap_model=umap_model_sample,verbose=True,min_topic_size=300,n_gram_range=(1,2))
    topics2, probs2 = topic_model_gpu.fit_transform(data[1:len(data)//4], embeddings_sample)
else:
    topic_model_gpu = BERTopic(umap_model=umap_model,verbose=True,min_topic_size=400,n_gram_range=(1,2))
    topics2, probs2 = topic_model_gpu.fit_transform(data[1:], embeddings)

2023-03-06 19:14:44,157 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-03-06 19:54:47,784 - BERTopic - Clustered reduced embeddings


In [24]:
!date

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Mar  6 20:07:17 UTC 2023


In [25]:
topic_model_gpu.get_topic_freq().head()

,Topic,Count
0,-1,2376067
1,0,39821
2,1,36256
3,2,33925
4,3,26524


In [ ]:
topic_model_gpu.visualize_topics()

In [32]:
hierarchical_topics = topic_model_gpu.hierarchical_topics(data[1:len(data)//4])
tree = topic_model_gpu.get_topic_tree(hierarchical_topics)
print(tree)

100%|██████████| 1450/1450 [05:38<00:00,  4.28it/s]


.
├─it_is_you_the_to
│    ├─fiverr_use_domain_links_backlinks
│    │    ├─fiverr_backlinks_pbn_pr_links
│    │    │    ├─keyword_tool_niche_keywords_keyword research
│    │    │    │    ├─niche_niches_the niche_niche is_your niche
│    │    │    │    │    ├─■──micro_micro niche_niche_niche sites_niche site ── Topic: 662
│    │    │    │    │    └─niche_niches_the niche_niche is_csv
│    │    │    │    │         ├─■──niche_niches_the niche_niche is_csv ── Topic: 81
│    │    │    │    │         └─■──niche_keywords_keyword_research_niche keyword ── Topic: 1007
│    │    │    │    └─keyword_tool_keywords_keyword research_the keyword
│    │    │    │         ├─keyword_density_keywords_keyword density_searches
│    │    │    │         │    ├─competition_competitor_the competition_competitors_competition is
│    │    │    │         │    │    ├─■──competition_competitor_competitors_the competition_your competitor ── Topic: 387
│    │    │    │         │    │    └─■──competition_keyword_low co

In [26]:
if sample:
  topic_model_gpu.save("sample_model")
else:
  topic_model_gpu.save("complex_model")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.8/site-packages/scipy/sparse/_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


OSError: [Errno 28] No space left on device

In [ ]:
np.__version__

In [ ]:
!pip show bertopic | grep Version

In [ ]:
!pip show tensorflow | grep Version

In [ ]:
!python --version 

In [ ]:
!pip show torch | grep Version

In [ ]:
#